In [ ]:
%run ../config/init.py

### Creating results folders

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'peak_calling'))
sample_table_file = os.path.join(DATA, DATASET, 'sample_table.csv')
sample_table = pandas.read_csv(sample_table_file, keep_default_na=False)
sample_table.head()

### Peak calling workflow using MACE

In [ ]:
log_file = 'peak_calling.log'

peakcalling_yml = {
    'chrom_size': {'class': 'File', 'path': GENOME_CHROMSIZES },
    'genome_gtf': {'class': 'File', 'path': GENOME_GTF },
    'tss_size': 1000,
    'norm_method': 'SNR',    
    'output_basename': [],
    'sorted_bam': []
}

for c in sample_table['condition'].unique():
    rep = sample_table[sample_table['condition'] == c]['sample_name']
    sorted_bam = []
    for r in rep:
        r = os.path.join(data_dir, r + '_sorted.bam')
        sorted_bam.append({'class': 'File', 'path': r, 'secondaryFiles': [{'class': 'File', 'path': r + '.bai'}]})
    if sorted_bam:
        peakcalling_yml['output_basename'].append(c)
        peakcalling_yml['sorted_bam'].append(sorted_bam)
    
if peakcalling_yml['output_basename']:
    write_to_yaml(peakcalling_yml, 'peakcalling.yml')  
    cmd_header = '{} {}/ChIP-exo/peak-caller-MACE-SE.cwl peakcalling.yml > {} 2>&1 &'.format(
        CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)